In [1]:
!pip install datasets
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 14.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [2]:
from datasets import load_dataset
import torch

In [3]:
emotions = load_dataset("emotion")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [4]:
emotions

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [5]:
train_ds = emotions['train']
train_ds

Dataset({
    features: ['text', 'label'],
    num_rows: 16000
})

In [6]:
len(train_ds)

16000

In [7]:
train_ds[:3]

{'text': ['i didnt feel humiliated',
  'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake',
  'im grabbing a minute to post i feel greedy wrong'],
 'label': [0, 0, 3]}

In [8]:
emotions.set_format('pandas')

df = emotions['train'][:]
df.head()

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3


In [9]:
def label_int2str(row):
  return emotions['train'].features['label'].int2str(row)

df['label_name'] = df['label'].apply(label_int2str)
df.head()

,text,label,label_name
0,i didnt feel humiliated,0,sadness
1,i can go from feeling so hopeless to so damned...,0,sadness
2,im grabbing a minute to post i feel greedy wrong,3,anger
3,i am ever feeling nostalgic about the fireplac...,2,love
4,i am feeling grouchy,3,anger


In [10]:
num_labels = df.label_name.nunique()

In [11]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
model_ckpt = 'distilbert-base-cased'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

cpu


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [12]:
encoded_text = tokenizer('tokenize the text is the core task in NLP')
encoded_text

{'input_ids': [101, 22559, 3708, 1103, 3087, 1110, 1103, 4160, 4579, 1107, 21239, 2101, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [13]:
tokens = tokenizer.convert_ids_to_tokens(encoded_text.input_ids)
print(tokens)

['[CLS]', 'token', '##ize', 'the', 'text', 'is', 'the', 'core', 'task', 'in', 'NL', '##P', '[SEP]']


In [14]:
#serevral metadata about the tokenizer
print("The vocabulary size is :", tokenizer.vocab_size)
print("Maximum Context size :", tokenizer.model_max_length)
print("Name of fields model need in the forwards pass:", tokenizer.model_input_names)

The vocabulary size is : 28996
Maximum Context size : 512
Name of fields model need in the forwards pass: ['input_ids', 'attention_mask']


In [15]:
#tokenize the entire dataset now
def tokenize(batch):
  print(type(batch["text"]))
  return tokenizer(batch["text"], padding = True, truncation = True)

In [16]:
emotions.reset_format()
tokenize(emotions['train'][:1])

<class 'list'>


{'input_ids': [[101, 178, 1238, 1204, 1631, 21820, 21896, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1]]}

In [17]:
emotions_encoded = emotions.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

<class 'list'>


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

<class 'list'>


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

<class 'list'>


In [18]:
model = (AutoModelForSequenceClassification.from_pretrained(model_ckpt,num_labels=num_labels).to(device))

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
#let's create the compute matrics which will return the F1 score and accuracy score

from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  f1 = f1_score(labels,preds, average="weighted")
  acc = accuracy_score(labels, preds)

  return {"accuracy": acc, "f1":f1}

In [20]:
from transformers import Trainer , TrainingArguments

batch_size = 64
logging_steps = len(emotions_encoded["train"]) // batch_size
model_name = f"{model_ckpt}-finetuned-emotion"

training_args = TrainingArguments(

                                  output_dir = model_name,
                                  num_train_epochs = 2,
                                  learning_rate = 2e-5,
                                  per_device_train_batch_size = batch_size,
                                  per_device_eval_batch_size = batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy = "epoch",
                                  logging_steps=logging_steps,
)

In [21]:
trainer = Trainer(model = model,
    args = training_args,
    compute_metrics = compute_metrics,
    train_dataset = emotions_encoded['train'],
    eval_dataset = emotions_encoded['validation'],
    tokenizer = tokenizer
)


In [ ]:
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.749300,0.262919,0.922000,0.921866


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.749300,0.262919,0.922000,0.921866


In [27]:
preds_output = trainer.predict(emotions_encoded["validation"])

In [28]:
preds_output.metrics

{'test_loss': 0.21775516867637634,
 'test_accuracy': 0.9155,
 'test_f1': 0.9154885594267999,
 'test_runtime': 222.185,
 'test_samples_per_second': 9.002,
 'test_steps_per_second': 0.144}

In [ ]:
# trainer.save_model("./{}".format(model_name))

In [ ]:
# from transformers import DistilBertConfig, DistilBertModel
# path = 'path_to_my_model'
# model = DistilBertModel.from_pretrained(path)